## Assignment 1
# Group 13
Mathieu Mailhot - Isabel Lougheed - Frank-Lucas Pantazis

Imports

In [1]:
import matplotlib
import numpy as np

In [2]:
"""
- CKD: 28 numerical features, 1 target binary classification variable ("Normal" / "CKD")
- Battery: 32 real-valued features, 2 classes ("Normal" / "Defective")
"""

# load data sets

# statistical analysis on the datasets

# - normalize

# models: all features, selective features based on statistical analysis (dropping features)

'\n- CKD: 28 numerical features, 1 target binary classification variable ("Normal" / "CKD")\n- Battery: 32 real-valued features, 2 classes ("Normal" / "Defective")\n'